In [1]:
import os, glob
import datetime

import xmltodict as xd

import numpy as np
import pandas as pd
import h5py

import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing

# lets make a little data set for fun...
mh_dir = os.path.abspath('../db/mh_data/')
mh_cases = glob.glob(os.path.join(mh_dir, '*'))

class Input: # an input struct
    pass

In [2]:
def xml_parser(xml_path):

    with open(xml_path) as fd:
            doc = xd.parse(fd.read())
            fd.close()

    raw_db = doc['anaesthetic']['data']['var']
    print("FILE READ")


    for i in raw_db[3:]:
        name = i['vaname']

        times = str(i['vatimes']).replace('None','000000').split(',')
        values = str(i['vavalues']).replace('NA','nan').split(',')

        times = np.asarray(times)
        values = np.asarray(values).astype('float')

        var_df = pd.DataFrame(data = {'time' : times, name : values})

        if 'full_df' in locals():
            full_df = full_df.join(var_df.set_index('time'), on='time')
        else:
            full_df = var_df

    print("XML PARSED")

    return full_df

def delta_spo2(spo2_arr):
    # compute the difference between the maximum value 
    max_val = max(spo2_arr)
    min_val = min(spo2_arr)
    d = max_val - min_val

    return d

# a sample will be 6 entries (=60 seconds) of every datapoint to determine if
# there will be a change in spo2 in the next 60 seconds
# spo2, hr, 
def data_generator(patient_df, threshold):
    # slice the df into array of 6 element dfs
    interval_df = []

    for i in range(patient_df.shape[0]):
        if (i+1) % 6 == 0:
            # split every 6 timestamp (60 seconds)
            a = i - 5
            interval_df.append(patient_df[a:i+1])
        else:
            continue

    # compute spo2 delta
    for i in range(len(interval_df)):
        sample = Input()
        sample.x = np.asarray(interval_df[i].unstack()) # vector of input data from 
        try:
            sample.d = delta_spo2(interval_df[i+1]['spo2.SpO2'])
        except:
            print("end of dataset")
            break
        # label
        if sample.d > threshold:
            sample.y = 1
        else:
            sample.y = 0 
        db.append(sample)

    return db


In [17]:
db = [] # list of all input structs
# parse every xml file and save each to a separate h5 file for future use
# spo2.SpO2, co2.et, ecg.hr, nibp.sys, nibp.dia
def mk_5npy():
    for i in mh_cases:
        print(i)
        df = xml_parser(i)

        # for all features simply use df
        # spo2.SpO2, co2.et, ecg.hr, nibp.sys, nibp.dia
        df2 = pd.DataFrame(df,
                columns=['ecg.hr',
                    'co2.et', 'nibp.sys',
                    'nibp.dia', 'spo2.SpO2']
                )

        df2 = df2[np.abs(df2-df2.mean()) <= (3*df2.std())]
        df2 = df2.dropna()

        # scale the values between 1-0 the data by patient....
        x = df2.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df2 = pd.DataFrame(x_scaled, columns=df2.columns)

        data_generator(df2, 0.011)

    X = []
    Y = []

    for i in db:
        X.append(i.x)
        Y.append(i.y)

    X = np.asarray(X).astype('float')
    Y = np.asarray(Y).astype('int')
    print("stable: " + str(np.sum(Y == 0)))
    print("unstable: " + str(np.sum(Y == 1)))

    #np.save("../db/x5.npy", X)
    #np.save("../db/y5.npy", Y)
    return df2, X, Y

example_df1, X1, Y1 = mk_5npy()

/home/max/src/dl-anesthesia/db/mh_data/18_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/24_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/25_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/17_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/30_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/20_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/26_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/19_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/22_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/23_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/21_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/28_2.xml
FILE READ
XML PARSED
end of 

In [16]:
db = [] # list of all input structs
# parse every xml file and save each to a separate h5 file for future use
# spo2.SpO2, co2.et, ecg.hr, nibp.sys, nibp.dia
def mk_51npy():
    for i in mh_cases:
        print(i)
        df = xml_parser(i)

        # for all features simply use df
        df2 = df.drop(['time'], axis=1)
        df2 = df2[np.abs(df2-df2.mean()) <= (3*df2.std())]
        df2 = df2.dropna()

        # scale the values between 1-0 the data by patient....
        x = df2.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df2 = pd.DataFrame(x_scaled, columns=df2.columns)

        data_generator(df2, 0.017)

    X = []
    Y = []

    for i in db:
        X.append(i.x)
        Y.append(i.y)

    X = np.asarray(X).astype('float')
    Y = np.asarray(Y).astype('int')
    print("stable: " + str(np.sum(Y == 0)))
    print("unstable: " + str(np.sum(Y == 1)))
    print(X.shape)
    print(Y.shape)

    #np.save("../db/x51.npy", X)
    #np.save("../db/y51.npy", Y)
    return df2, X, Y

example_df2, X2, Y2 = mk_51npy()

/home/max/src/dl-anesthesia/db/mh_data/18_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/24_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/25_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/17_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/30_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/20_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/26_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/19_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/22_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/23_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/21_2.xml
FILE READ
XML PARSED
end of dataset
/home/max/src/dl-anesthesia/db/mh_data/28_2.xml
FILE READ
XML PARSED
end of 

# Demonstration Dataset

For this presentation I will be using the open source anesthesia dataset from the University of Auckland (
Physiological changes during anaesthesia for surgery with potential for moderate blood loss - https://researchspace.auckland.ac.nz/handle/2292/10357). It contains physiological recordings from 13 procedures which were stored in separate XML files. The data was recorded every 10 seconds.

**GOAL:** Predict change in oxygen saturation using the past minute of physiological data

We will create two sets:
- a 5 variable dataset where a Spo2 change of more 1.1% will be considered unstable
- a 51 variable dataset where a Spo2 change of more 1.7% will be considered unstable

This is a binary classification problem (stable vs unstable).

Note: the difference in threshold for unstability is simply to ensure that the dataset are evenly split between stable vs unstable.

Important things to consider when preparing data:
- outliers
- missing data
- scaling/normalization

## The 5 Variable Dataset

Each input sample will consist of a minute (6 timesteps) of physiological recording of 5 variables which results in a tensor of $5\cdot6=30$ features.

In [31]:
print(example_df1.head())
print(example_df1.columns)
print("\n")
print(X1.shape)
print(Y1.shape)
print("\n")
print("stable: " + str(np.sum(Y1 == 0)))
print("unstable: " + str(np.sum(Y1 == 1)))

     ecg.hr    co2.et  nibp.sys  nibp.dia  spo2.SpO2
0  0.487805  0.392727  0.551761  0.436792   0.989547
1  0.536585  0.000000  0.551761  0.436792   1.000000
2  0.536585  0.327273  0.551761  0.436792   0.972125
3  0.512195  0.283636  0.551761  0.436792   0.968641
4  0.487805  0.516364  0.551761  0.436792   0.951220
Index(['ecg.hr', 'co2.et', 'nibp.sys', 'nibp.dia', 'spo2.SpO2'], dtype='object')


(3559, 30)
(3559,)


stable: 1766
unstable: 1793


## The 51 Variable Dataset

Each input sample will consist of a minute (6 timesteps) of physiological recording of 51 variables which results in a tensor of $51\cdot6=306$ features.

In [29]:
print(example_df2.head())
print(example_df2.columns)
print("\n")
print(X2.shape)
print(Y2.shape)
print("\n")
print("stable: " + str(np.sum(Y2 == 0)))
print("unstable: " + str(np.sum(Y2 == 1)))

     ecg.hr   ecg.st1  ecg.st2  ecg.st3  ecg.imp_rr    p1.sys    p1.dia  \
0  0.085714  0.634921      0.0      0.0         0.0  0.290537  0.092306   
1  0.085714  0.761905      0.0      0.0         0.0  0.280308  0.087422   
2  0.085714  0.746032      0.0      0.0         0.0  0.315123  0.084246   
3  0.085714  0.595238      0.0      0.0         0.0  0.273841  0.086522   
4  0.085714  0.738095      0.0      0.0         0.0  0.265171  0.084625   

    p1.mean  p1.hr  p2.sys  ...     aa.fi  aa.mac_sum  p5.sys  p5.dia  \
0  0.149849  0.188     0.0  ...  1.000000     1.00000     0.0     0.0   
1  0.144809  0.184     0.0  ...  0.979592     0.87500     0.0     0.0   
2  0.142642  0.184     0.0  ...  1.000000     0.75000     0.0     0.0   
3  0.141229  0.184     0.0  ...  0.877551     0.81250     0.0     0.0   
4  0.138308  0.184     0.0  ...  0.877551     0.78125     0.0     0.0   

   p5.mean  p5.hr  p6.sys  p6.dia  p6.mean  p6.hr  
0      0.0    0.0     0.0     0.0      0.0    0.0  
1     